In this data preparation part, the goal is to clean up the data to ensure the algorithms can run more smoothly.  To do that, we will 
1. Handle Basic Transformation
2. Handle Duplication
2. Convert Data Types
3. Handle Incorrect Values
4. Handle Missing Values

### 0. Load Relevant Libraries and Dataset

In [464]:
# load relevant libraries
import pandas as pd
import numpy as np

# load sklearn for baseline model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor


In [465]:
# load the dataset

df=pd.read_csv('Sephora_Table.csv')

In [466]:
df.shape

(2836, 28)

### 1. Basic Transformation

In [467]:
# Get ratio of stars to number of reviews

stars=['five_star', 'four_star', 'three_star', 'two_star', 'one_star']

for star in stars:
    df[star]=df[star]/df['number_of_reviews']

We notice that the number of stars by themselves doesn't really matter, as we may have 10 five star reviews but 100 one star reviews.  So we try to divide number of stars by total number of reviews.

### 2. Handle Duplication

In [468]:
# Find out how many rows are duplicated

df.duplicated().sum()

125

In [469]:
df[df.duplicated()]

,product_links,brand_name,product_name,item_id,product_type_1,product_type_2,number_of_reviews,number_of_loves,price,skin_type_normal,...,skin_concerns_dullness_uneven,skin_concerns_dryness,skin_concerns_oiliness,skin_concerns_puffiness,skin_concerns_redness_irritation,five_star,four_star,three_star,two_star,one_star
819,https://www.sephora.com/ca/en/product/facial-t...,SK-II,Facial Treatment Essence (Pitera Essence),1850346.0,Moisturizers,Mists & Essences,985.0,86094.0,241.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.556345,0.151269,0.090355,0.077157,0.124873
880,https://www.sephora.com/ca/en/product/rose-flo...,FRESH,Rose Floral Toner,661157.0,Cleansers,Toners,897.0,70748.0,53.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.545151,0.210702,0.108138,0.068004,0.068004
889,https://www.sephora.com/ca/en/product/make-up-...,CAUDALIE,Make-Up Removing Cleansing Oil,2062503.0,Cleansers,Face Wash & Cleansers,277.0,23960.0,34.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.664260,0.148014,0.090253,0.057762,0.039711
892,https://www.sephora.com/ca/en/product/squalane...,BIOSSANCE,Squalane + Antioxidant Cleansing Oil,6.0,Cleansers,Face Wash & Cleansers,397.0,27038.0,40.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.622166,0.148615,0.075567,0.075567,0.078086
902,https://www.sephora.com/ca/en/product/micro-es...,ESTÉE LAUDER,Micro Essence Skin Activating Treatment Lotion,1641323.0,Moisturizers,Mists & Essences,68.0,19666.0,124.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.779412,0.132353,0.000000,0.029412,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715,https://www.sephora.com/ca/en/product/future-s...,SHISEIDO,Future Solution LX Eye and Lip Contour Regener...,0.0,Eye Care,Eye Creams & Treatments,28.0,4184.0,202.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.500000,0.214286,0.107143,0.071429,0.107143
2734,https://www.sephora.com/ca/en/product/lip-mask...,SEPHORA COLLECTION,Hydrating Lip Mask,1.0,Lip Treatments,Lip Balms & Treatments,232.0,19232.0,4.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.288793,0.185345,0.168103,0.137931,0.219828
2735,https://www.sephora.com/ca/en/product/orchidee...,GUERLAIN,Orchidee Imperiale Anti-Aging Eye & Lip Contou...,0.0,Eye Care,Eye Creams & Treatments,8.0,1351.0,258.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.500000,0.250000,0.000000,0.000000,0.250000
2767,https://www.sephora.com/ca/en/product/glopro-l...,BEAUTYBIO,GloPRO® LIP MicroTip™ Attachment Head,2163061.0,High Tech Tools,Anti-Aging,6.0,3680.0,46.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.666667,0.000000,0.000000,0.000000,0.333333


In [470]:
# remove duplicated rows

df=df.drop_duplicates()

In [471]:
df.duplicated().sum()

0

### 3. Handle Incorrect Values

By looking at the dataframe, we get that item_id is sometimes sourced incorrectly.  Luckily, as this information is not very relevant to our modelling, we will remove this column.

As product links and product name are not relevant either, we will also remove them

In [472]:
# drop item_id column

to_drop=['item_id', 'product_links', 'product_name']

df=df.drop(columns=to_drop)

In [473]:
df.head()

,brand_name,product_type_1,product_type_2,number_of_reviews,number_of_loves,price,skin_type_normal,skin_type_dry,skin_type_combination,skin_type_oily,...,skin_concerns_dullness_uneven,skin_concerns_dryness,skin_concerns_oiliness,skin_concerns_puffiness,skin_concerns_redness_irritation,five_star,four_star,three_star,two_star,one_star
0,DRUNK ELEPHANT,Moisturizers,Moisturizers,4504.0,263031.0,89.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.579929,0.123668,0.092140,0.092362,0.111901
1,TATCHA,Moisturizers,Moisturizers,3100.0,237318.0,89.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.596129,0.133871,0.101290,0.076129,0.092581
2,TATCHA,Moisturizers,Moisturizers,1475.0,119147.0,89.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.665763,0.107119,0.078644,0.079322,0.069153
3,LA MER,Moisturizers,Moisturizers,663.0,75093.0,235.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.588235,0.117647,0.081448,0.081448,0.131222
4,SK-II,Moisturizers,Mists & Essences,985.0,86094.0,241.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.556345,0.151269,0.090355,0.077157,0.124873


### 4. Handle Missing Values

In [474]:
df.shape

(2711, 25)

In [475]:
# Finding out portion of null value in each column

df.isnull().mean().sort_values(ascending=True)


brand_name                           0.002582
product_type_2                       0.002582
price                                0.002582
skin_concerns_redness_irritation     0.002951
skin_concerns_puffiness              0.002951
skin_concerns_oiliness               0.002951
skin_concerns_dryness                0.002951
skin_concerns_dullness_uneven        0.002951
skin_concerns_firmness_elasticity    0.002951
skin_concerns_dark_spots             0.002951
skin_concerns_fine_lines_wrinkles    0.002951
skin_concerns_pores                  0.002951
skin_type_oily                       0.002951
skin_type_combination                0.002951
skin_type_dry                        0.002951
skin_type_normal                     0.002951
number_of_loves                      0.002951
skin_type_sensitive                  0.002951
product_type_1                       0.022870
two_star                             0.036518
number_of_reviews                    0.036518
five_star                         

There are three cases of missing reviews:
1. Price, number of loves, product name, and brand name:  missing if the link is no longer working
2. Skin type and concerns:  missing due to sourcing issues
3. Review numbers:  missing due to no reviews
4. Product type 1:  missing due to unknown reason

For 1. and 2., as this means that data is missing completely at random and the missing portion is small (0.8%), we will choose to remove all the rows with such missing values in this case.  This method is called the complete-case analysis (CCA).

For 3., as this means that there are no reviews, we will put the numbers as 0.

For 4., we will use a new variable (='Missing') to indicate the value is missing, as we do not know if there is any importance of missingness, and we'd like to capture it if there is one.

In [476]:
# Imputing missing values in case 1 and 2 with CCA
df = df.dropna(subset=['price','skin_concerns_redness_irritation'])

In [477]:
df.isnull().mean().sort_values(ascending=True)


brand_name                           0.000000
skin_concerns_redness_irritation     0.000000
skin_concerns_puffiness              0.000000
skin_concerns_oiliness               0.000000
skin_concerns_dryness                0.000000
skin_concerns_dullness_uneven        0.000000
skin_concerns_firmness_elasticity    0.000000
skin_concerns_dark_spots             0.000000
skin_concerns_fine_lines_wrinkles    0.000000
skin_type_sensitive                  0.000000
skin_concerns_pores                  0.000000
skin_type_combination                0.000000
skin_type_dry                        0.000000
skin_type_normal                     0.000000
price                                0.000000
number_of_loves                      0.000000
product_type_2                       0.000000
skin_type_oily                       0.000000
product_type_1                       0.020348
two_star                             0.033666
number_of_reviews                    0.033666
five_star                         

In [478]:
# Imputing missing values in case 3 with 0

number_of_reviews_cols=['five_star', 'four_star',
       'three_star', 'two_star', 'one_star', 'number_of_reviews']

for reviews_number in number_of_reviews_cols:
    df[reviews_number].fillna(0, inplace=True)

In [479]:
df.isnull().mean().sort_values(ascending=True)


brand_name                           0.000000
three_star                           0.000000
four_star                            0.000000
five_star                            0.000000
skin_concerns_redness_irritation     0.000000
skin_concerns_puffiness              0.000000
skin_concerns_oiliness               0.000000
skin_concerns_dryness                0.000000
skin_concerns_dullness_uneven        0.000000
skin_concerns_firmness_elasticity    0.000000
skin_concerns_dark_spots             0.000000
two_star                             0.000000
skin_concerns_pores                  0.000000
skin_type_sensitive                  0.000000
skin_type_oily                       0.000000
skin_type_combination                0.000000
skin_type_dry                        0.000000
skin_type_normal                     0.000000
price                                0.000000
number_of_loves                      0.000000
number_of_reviews                    0.000000
product_type_2                    

In [480]:
# Imputing missing values in case 4 with value = 'Missing'

df['product_type_1'] = df['product_type_1'].astype('object')

df['product_type_1'].fillna('Missing', inplace=True)


In [481]:
df.isnull().mean().sort_values(ascending=True)


brand_name                           0.0
three_star                           0.0
four_star                            0.0
five_star                            0.0
skin_concerns_redness_irritation     0.0
skin_concerns_puffiness              0.0
skin_concerns_oiliness               0.0
skin_concerns_dryness                0.0
skin_concerns_dullness_uneven        0.0
skin_concerns_firmness_elasticity    0.0
skin_concerns_dark_spots             0.0
two_star                             0.0
skin_concerns_pores                  0.0
skin_type_sensitive                  0.0
skin_type_oily                       0.0
skin_type_combination                0.0
skin_type_dry                        0.0
skin_type_normal                     0.0
price                                0.0
number_of_loves                      0.0
number_of_reviews                    0.0
product_type_2                       0.0
product_type_1                       0.0
skin_concerns_fine_lines_wrinkles    0.0
one_star        

### 5. Convert Data Types

In [482]:
df.dtypes

brand_name                            object
product_type_1                        object
product_type_2                        object
number_of_reviews                    float64
number_of_loves                      float64
price                                float64
skin_type_normal                     float64
skin_type_dry                        float64
skin_type_combination                float64
skin_type_oily                       float64
skin_type_sensitive                  float64
skin_concerns_fine_lines_wrinkles    float64
skin_concerns_pores                  float64
skin_concerns_dark_spots             float64
skin_concerns_firmness_elasticity    float64
skin_concerns_dullness_uneven        float64
skin_concerns_dryness                float64
skin_concerns_oiliness               float64
skin_concerns_puffiness              float64
skin_concerns_redness_irritation     float64
five_star                            float64
four_star                            float64
three_star

In [483]:
df.columns

Index(['brand_name', 'product_type_1', 'product_type_2', 'number_of_reviews',
       'number_of_loves', 'price', 'skin_type_normal', 'skin_type_dry',
       'skin_type_combination', 'skin_type_oily', 'skin_type_sensitive',
       'skin_concerns_fine_lines_wrinkles', 'skin_concerns_pores',
       'skin_concerns_dark_spots', 'skin_concerns_firmness_elasticity',
       'skin_concerns_dullness_uneven', 'skin_concerns_dryness',
       'skin_concerns_oiliness', 'skin_concerns_puffiness',
       'skin_concerns_redness_irritation', 'five_star', 'four_star',
       'three_star', 'two_star', 'one_star'],
      dtype='object')

We know that 
1. number_of_reviews and number_of_loves are integers,
2. skin types and skin concerns should be boolean values, 
3. product types should be categories

let's update the above

In [484]:
skin_type_cols=['skin_type_normal', 'skin_type_dry',
       'skin_type_combination', 'skin_type_oily', 'skin_type_sensitive']
skin_concern_cols=['skin_concerns_fine_lines_wrinkles', 'skin_concerns_pores',
       'skin_concerns_dark_spots', 'skin_concerns_firmness_elasticity',
       'skin_concerns_dullness_uneven', 'skin_concerns_dryness',
       'skin_concerns_oiliness', 'skin_concerns_puffiness',
       'skin_concerns_redness_irritation']

for skin_type in skin_type_cols:
    df[skin_type] = df[skin_type].astype('bool')
    
for skin_concern in skin_concern_cols:
    df[skin_concern] = df[skin_concern].astype('bool')

    

In [485]:
product_type_cols=['product_type_1', 'product_type_2']
for product_type in product_type_cols:
    df[product_type] = df[product_type].astype('category')


In [486]:
reviews_cols=['number_of_reviews']

for review_type in reviews_cols:
    df[review_type] = df[review_type].astype('int64')


### 6. Categorical Encoding

In [487]:
X_train, X_test, y_train, y_test = train_test_split(
    df,  # predictors
    df['number_of_loves'],  # target
    test_size=0.3,
    random_state=0)

In [488]:
categorical_cols=[]
for variables in df.columns:
    if df[variables].dtype=='object':
        categorical_cols.append(variables)
        
categorical_cols

['brand_name']

In [489]:
categorical_cols=['brand_name','product_type_1' ,'product_type_2' ]

In [490]:
for col in categorical_cols:
# let's obtain the counts for each one of the labels
# in the variable Neigbourhood

    count_map = X_train[col].value_counts().to_dict()

    X_train[col] = X_train[col].map(count_map)
    X_test[col] = X_test[col].map(count_map)
    

/Users/audreytang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/audreytang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [491]:
# for variables in test set that didn't appear in the training set
# the above step will give them the null value
# use fillna again to ensure the values can be passed in the model

X_test['brand_name'].fillna(0, inplace=True)
X_test['product_type_2'].fillna(0, inplace=True)


/Users/audreytang/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [492]:
X_test.isnull().sum()


brand_name                           0
product_type_1                       0
product_type_2                       0
number_of_reviews                    0
number_of_loves                      0
price                                0
skin_type_normal                     0
skin_type_dry                        0
skin_type_combination                0
skin_type_oily                       0
skin_type_sensitive                  0
skin_concerns_fine_lines_wrinkles    0
skin_concerns_pores                  0
skin_concerns_dark_spots             0
skin_concerns_firmness_elasticity    0
skin_concerns_dullness_uneven        0
skin_concerns_dryness                0
skin_concerns_oiliness               0
skin_concerns_puffiness              0
skin_concerns_redness_irritation     0
five_star                            0
four_star                            0
three_star                           0
two_star                             0
one_star                             0
dtype: int64

### 6. Baseline Model

In [493]:
X_train=X_train.drop(columns='number_of_loves')
X_test=X_test.drop(columns='number_of_loves')


After our first round of data preprocessing, let's get a baseline model.  We choose random forests as it is a relatively simple model that gives high performance nonetheless without overfitting

In [494]:
# create a function to build random forests and compare performance in train and test set


def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests R2: {}'.format(r2_score(y_train, pred)))
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests R2: {}'.format(r2_score(y_test, pred)))


In [495]:
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests R2: 0.7747549489577906
Test set
Random Forests R2: 0.6815979112685899


### 5. Save Dataframe with New Name

In [496]:
df.to_csv('Sephora Dataset Preprocessed', index=False)